In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.mgmt.web import WebSiteManagementClient
from azure.keyvault.secrets import SecretClient
import requests
import pprint

In [ ]:
tenant_id = "5878df98-6f88-48ab-9322-998ce557088d"
subscription_id = "ff442a29-fc06-4a13-8e3e-65fd5da513b3"
subscription = "pins-odw-data-dev-sub"
resource_group_name = "pins-rg-function-app-odw-dev-uks"
function_app_name = "pins-fnapp01-odw-dev-uks"
function_name = "caseschedule"

In [ ]:
credential = DefaultAzureCredential(authority=f"https://login.microsoftonline.com/{tenant_id}")
token = credential.get_token("https://management.azure.com/.default")
access_token = token.token

In [ ]:
functions_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.Web/sites/{function_app_name}/functions?api-version=2022-03-01"

headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

In [ ]:
def listfunctions():
    function_list = []
    functions = requests.get(functions_url, headers=headers)
    for function in functions.json()['value']:
        function_name = function['properties']['config']['name']
        function_list.append(function_name)
    return function_list

listfunctions()

In [ ]:
def getfunctionurls():
    urls = []
    for function in listfunctions():
        function_key_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{resource_group_name}/providers/Microsoft.Web/sites/{function_app_name}/functions/{function}/listkeys?api-version=2022-03-01"
        keys = requests.post(function_key_url, headers=headers)
        code = keys.json()['default']
        function_url = f"https://{function_app_name}.azurewebsites.net/api/{function}?code={code}"
        urls.append(function_url)
    return urls

getfunctionurls()

In [ ]:
def generatesecretnames():
    secret_names = []
    for function in listfunctions():
        secret_name = f"function-url-{function}"
        secret_names.append(secret_name)
    return secret_names

generatesecretnames()


In [ ]:
names = generatesecretnames()
urls = getfunctionurls()

keyvault_dict = {name: url for name, url in zip(names, urls)}

pprint.pprint(keyvault_dict)